# Many to One

In [1]:
import torch
import numpy as np
import gc

device = 'mps'

In [2]:
class SimpleRNN(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.RNN(input_size=self.I,
                                hidden_size=self.H,
                                nonlinearity='tanh',
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0 = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])

        return out
    
class LSTM(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.lstm = torch.nn.LSTM(input_size=self.I,
                                 hidden_size=self.H,
                                 batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)
        c0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, (h_0, c_0) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out
    
class GRU(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.gru = torch.nn.GRU(input_size=self.I,
                                hidden_size=self.H,
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0= self.gru(x, h0)
        out = self.fc(out[:, -1, :])

        return out

In [3]:
rnn_model = SimpleRNN(n_input=1, n_hidden=40, n_output=1).to(device)
lstm_model = LSTM(n_input=1, n_hidden=40, n_output=1).to(device)
gru_model = GRU(n_input=1, n_hidden=40, n_output=1).to(device)

In [5]:
X = np.random.randint(1, 15, size=(50000, 6, 1)) # 입력 데이터 생성, 1~14 사이의 정수 50000개를 랜덤하게 생성
Y = np.array([np.sum(x) for x in X]) # 타겟 데이터 생성, 모든 입력 데이터의 총합이 타겟

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 타겟 데이터를 텐서로 변환

criterion = torch.nn.MSELoss()
rnn_optimizer = torch.optim.Adam(rnn_model.parameters())
lstm_optimizer = torch.optim.Adam(lstm_model.parameters())
gru_optimizer = torch.optim.Adam(gru_model.parameters())

In [6]:
for epoch in range(4000):
    outputs = rnn_model(X)
    loss = criterion(outputs.squeeze(), Y)

    rnn_optimizer.zero_grad()
    loss.backward()
    rnn_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0], [4.0], [6.0], [8.0], [10.0], [11.0]]], dtype=torch.float32).to(device)
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {X_test.squeeze().tolist()}")
output = round(rnn_model(X_test).item(), 1)
answer = round(sum(X_test.squeeze().tolist()), 1)

print(f"Output: {output}, 정답: {answer}")

Epoch [100/4000], Loss: 1588.6315
Epoch [200/4000], Loss: 1279.7161
Epoch [300/4000], Loss: 1031.8058
Epoch [400/4000], Loss: 828.5258
Epoch [500/4000], Loss: 662.2735
Epoch [600/4000], Loss: 527.5439
Epoch [700/4000], Loss: 419.7122
Epoch [800/4000], Loss: 334.6778
Epoch [900/4000], Loss: 268.7424
Epoch [1000/4000], Loss: 218.5683
Epoch [1100/4000], Loss: 181.1699
Epoch [1200/4000], Loss: 153.9168
Epoch [1300/4000], Loss: 134.5372
Epoch [1400/4000], Loss: 121.1149
Epoch [1500/4000], Loss: 112.0772
Epoch [1600/4000], Loss: 106.1717
Epoch [1700/4000], Loss: 102.4336
Epoch [1800/4000], Loss: 100.1453
Epoch [1900/4000], Loss: 98.7928
Epoch [2000/4000], Loss: 98.0217
Epoch [2100/4000], Loss: 97.5979
Epoch [2200/4000], Loss: 97.3722
Epoch [2300/4000], Loss: 97.2523
Epoch [2400/4000], Loss: 97.0127
Epoch [2500/4000], Loss: 87.2679
Epoch [2600/4000], Loss: 59.3742
Epoch [2700/4000], Loss: 36.2427
Epoch [2800/4000], Loss: 28.0536
Epoch [2900/4000], Loss: 21.9977
Epoch [3000/4000], Loss: 17.776